In [45]:
# Import Splinter, BeautifulSoup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

In [46]:
# Set the executable path and initialize Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\jmhua\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


In [47]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com/'
browser.visit(url)

# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [48]:
# Convert the browser html to a soup object and then quit the browser
html = browser.html
news_soup = soup(html, 'html.parser')

slide_elem = news_soup.select_one('div.list_text')

In [49]:
slide_elem.find('div', class_='content_title')

<div class="content_title">What's Mars Solar Conjunction, and Why Does It Matter?</div>

In [50]:
# Use the parent element to find the first a tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

"What's Mars Solar Conjunction, and Why Does It Matter?"

In [51]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'NASA spacecraft at Mars are going to be on their own for a few weeks when the Sun comes between Mars and Earth, interrupting communications.'

### Featured Images


In [52]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [53]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [54]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [55]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars3.jpg'

In [56]:
# Find the relative image url
img_url_rel2 = img_soup.find('img', class_='headerimage fade-in').get('src')
img_url_rel2

'image/featured/mars3.jpg'

In [57]:
# Use the base url to create an absolute url
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars3.jpg'

### Mars Facts

In [58]:
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.head()

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"


In [59]:
df.columns=['Description', 'Mars', 'Earth']
df.set_index('Description', inplace=True)
df

,Mars,Earth
Description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [60]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

### Scrape high-reso mars' hemisphere images and titles 

In [61]:
# 1. Use browser to visit the URL 
url = 'https://marshemispheres.com/'

browser.visit(url)

hemi_html=browser.html
hemi_soup = soup(hemi_html,'html.parser')

hemisphere_images = hemi_soup.find_all('div', class_='item')

In [62]:


# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []

# Create loop to scrape through all hemisphere information
for item in hemisphere_images:
    hemisphere = {}
    
    #find the title
    titles = item.find('h3').text
    
    #find the path for the corresponding title
    image_path = item.find('a', class_='itemLink product-item')['href']
    
    #scrape the image path to get the .jpg path
    browser.visit(url + image_path)
    
    # parse the data to get .jpg
    jpg_html = browser.html
    jpg_soup = soup(jpg_html, 'html.parser')
    jpg_image_path = jpg_soup.find('div', class_= 'downloads')
    image_url = jpg_image_path.find('a')['href']
    
    img_url = url + image_url 
    print(titles)
    print(img_url)
    
    # append list
    hemispheres = {'img_url': img_url,
            'title': titles}

    hemisphere_image_urls.append(hemispheres)
    
    browser.back()



Cerberus Hemisphere Enhanced
https://marshemispheres.com/images/full.jpg
Schiaparelli Hemisphere Enhanced
https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg
Syrtis Major Hemisphere Enhanced
https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg
Valles Marineris Hemisphere Enhanced
https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg


In [63]:
# 4. Print the list that holds the dictionary of each image url and title.
df=pd.DataFrame(hemisphere_image_urls)
df

,img_url,title
0,https://marshemispheres.com/images/full.jpg,Cerberus Hemisphere Enhanced
1,https://marshemispheres.com/images/schiaparell...,Schiaparelli Hemisphere Enhanced
2,https://marshemispheres.com/images/syrtis_majo...,Syrtis Major Hemisphere Enhanced
3,https://marshemispheres.com/images/valles_mari...,Valles Marineris Hemisphere Enhanced


In [ ]:
browser.quit()